<h3> Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

from plotly.subplots import make_subplots

<h3> Loading Data

In [2]:
df = pd.read_csv('../../data/(A) raw_data.csv', index_col=0)

<h3> Inspecting Data

In [3]:
df

,Age,Gender,Country,Purchase Amount,Purchase Date,Product Category
User ID,,,,,,
1,56.0,Female,USA,331.79,2021-11-21,Sports
2,69.0,Male,Australia,335.72,2022-03-05,Home & Kitchen
3,46.0,NaN,Germany,493.18,NaN,Books
4,32.0,Male,Germany,80.97,2023-06-08,Sports
5,60.0,Female,UK,21.75,NaN,Beauty
...,...,...,...,...,...,...
14996,21.0,Male,UK,427.07,2023-08-20,Clothing
14997,53.0,Other,UK,27.73,2021-07-23,Home & Kitchen
14998,65.0,Female,Canada,108.57,2022-02-21,Beauty


-> Missing values are observed.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 1 to 15000
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Age               13500 non-null  float64
 1   Gender            14250 non-null  object 
 2   Country           13800 non-null  object 
 3   Purchase Amount   13200 non-null  float64
 4   Purchase Date     13950 non-null  object 
 5   Product Category  14100 non-null  object 
dtypes: float64(2), object(4)
memory usage: 820.3+ KB


-> Purchase Date should be converted to have a data type of DateTime.

-> Categorical data should be encoded before clustering if the clustering algorithm only accepts numerical inputs.

-> The dimensionality is not high; thus, dimensionality reduction techniques like PCA are not really required before clustering.

<h3> Handling Missing Values

In [5]:
df.isna().sum()

Age                 1500
Gender               750
Country             1200
Purchase Amount     1800
Purchase Date       1050
Product Category     900
dtype: int64

In [6]:
print(f"Percentage of rows with missing value(s): {round(df.isna().any(axis=1).sum()/len(df) * 100, 2)}%")

Percentage of rows with missing value(s): 39.49%


In [7]:
df.dropna(inplace=True)

<h3> Handling "Purchase Date"

In [8]:
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])

In [9]:
df.isna().sum()

Age                 0
Gender              0
Country             0
Purchase Amount     0
Purchase Date       0
Product Category    0
dtype: int64

<h3> Handling Duplication

In [10]:
df.duplicated().sum()

0

-> No duplication issue

<h3> Handling Outlier

In [11]:
numerical_columns = df.select_dtypes(exclude=['object']).columns
fig = make_subplots(rows=1, cols=len(numerical_columns), subplot_titles=numerical_columns)
for i, col in enumerate(numerical_columns, start=1):
    fig.add_trace(
        go.Box(
            y=df[col],
            name=col
        ),
        row=1, col=i
    )
fig.update_layout(
    width=400 * len(numerical_columns),
    showlegend=False
)
fig.show()

-> No outlier

-> Additionally, since the numerical columns have varying ranges, scaling should be performed before clustering to avoid bias.

<h3> Saving Cleaned Data

In [12]:
df.to_csv('../../data/(B) cleaned_data.csv', index=True) # Includes "Purchase Date" for visualization purposes
df.drop(['Purchase Date'], axis=1).to_csv('../../data/(C) analytical_data.csv', index=True) # Excludes "Purchase Date" for clustering purposes

-> I have thought of transforming some columns to RFM features (Recency, Frequency, Monetary). However, concerning no duplication happens, it implies that each user only perform one purchase. Therefore, the "recency" and "frequency" features seem to be less useful.

-> For "recency", the primary use is to discover the loyalty of customers to the business. However, since no customer has returned for a second purchase, it means that all of them are potentially inactive customers, and therefore, the recency values are not vital: both the customer with the lowest recency value and the highest recency value have not returned for a second purchase (we can't identify customers with high recency value as loyal customers).

-> For frequency, everyone will have the same value of 1, and therefore, does not help in distinguishing between users nor clustering users into a few segments based on similarity.

-> I have thought of the use of "Purchase Date" to identify temporal behaviors of different segments of users as well. But then, I realize that this only works if there are returning customers so that we can notice the change in needs or requirements of customers when doing purchasing decisions.

-> Overall, I don't see any significance of including "Purchase Date" in the upcoming clustering processes, and therefore I exclude it from the analytical dataset that will be used by clustering models.